# Libraries + Torch check

In [ ]:
import pandas as pd
import requests
import json
from pathlib import Path
import re
from datetime import date
import time
import os
from fastai.tabular import *
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
import sklearn.metrics
from sklearn.model_selection import train_test_split
import math

In [22]:
import torch

In [23]:
torch.cuda.device(0)

In [4]:
torch.cuda.get_device_name(0)

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

# Data Retrieval

In [24]:
#cleaned_item_names = [name[:-10] for name in list(os.walk(r'data/wiki_prices'))[0][2]]

In [25]:
item_names = pd.read_csv('wiki_item_names', header=None,names=['name'])

In [26]:
item_names

,name
0,3rd age amulet
1,3rd age axe
2,3rd age bow
3,3rd age cloak
4,3rd age druidic cloak
...,...
3286,Zogre bones
3287,Zombie head (Treasure Trails)
3288,Zul-andra teleport
3289,Zulrah's scales


In [27]:
cleaned_item_names = [name[0].replace(' ', '%20') for name in item_names.values]

In [28]:
item_names.loc[0]

name    3rd age amulet
Name: 0, dtype: object

In [29]:
cleaned_item_names[0:5]

['3rd%20age%20amulet',
 '3rd%20age%20axe',
 '3rd%20age%20bow',
 '3rd%20age%20cloak',
 '3rd%20age%20druidic%20cloak']

In [30]:
#pd.Series(cleaned_item_names).to_csv('./data/cleaned_item_names', index=False, header=False)

In [31]:
len(cleaned_item_names)

3291

In [32]:
# cleaned_item_names

In [33]:
def get_item_df(item_name, sleppy_time=1):
    base_url = r"https://oldschool.runescape.wiki/api.php?action=query&prop=revisions&rvprop=content&format=json&titles=Module%3AExchange%2F"
    postfix_url = r"%2FData"
    full_url = f'{base_url}{item_name}{postfix_url}'
    resp = requests.get(full_url)
    jsond = resp.json()
    try:
        regex_text = jsond['query']['pages'][next(iter(jsond['query']['pages'].keys()))]['revisions'][0]['*']
        #get all of the ones that have volume as well as price
        m = re.findall(r'[\d]+:[\d]+:[\d]+', regex_text)
        item_df = pd.DataFrame()
        item_df['dt'] = [int(string.split(':')[0]) for string in m]
        item_df[f'{item_name} price'] = [int(string.split(':')[1]) for string in m]
        item_df[f'{item_name} volume'] = [int(string.split(':')[2]) for string in m]
        item_df['date_obj'] = item_df['dt'].apply(lambda x: date.fromtimestamp(x))
        #take the ones that just have price so we can get the most recent price as well
        n = re.findall(r'[\d]+:[\d]+', regex_text)
        last_time = date.fromtimestamp(int(n[-1].split(':')[0]))
        last_price = int(n[-1].split(':')[1])
        item_df.loc[-1, 'date_obj'] = last_time
        item_df.loc[-1, f'{item_name} price'] = last_price
        item_df = item_df.fillna(method='ffill')
        item_df.set_index('date_obj', inplace=True)
        item_df = item_df.drop('dt', axis=1)
        return item_df
    except:
        pass



In [34]:
def retrieve_dfs(xslice = True):
    if xslice:
        item_dfs = [get_item_df(item_name,1) for item_name in cleaned_item_names[40:45]]
    else:
        item_dfs = [get_item_df(item_name,1) for item_name in cleaned_item_names]
    return item_dfs

In [ ]:
item_dfs = retrieve_dfs(False)

In [ ]:
len(item_dfs)

In [ ]:
item_dfs[0]

In [ ]:
def save_dfs(item_dfs):
    for df in item_dfs:
        try:
            df.to_csv(f'./data/wiki_prices/{df.columns[0]}.csv')
        except:
            pass

In [ ]:
save_dfs(item_dfs)

In [ ]:
def load_dfs(drop_blank=True):
    item_dfs = []
    for name in list(os.walk('./data/wiki_prices'))[0][2]:
        df = pd.read_csv(f'./data/wiki_prices/{name}')
        df = df.set_index(df.columns[0])
#         if drop_blank:
#             if df.empty:
#                 item_dfs.append(df)
#         else:
#             item_dfs.append(df)
        item_dfs.append(df)
    return item_dfs

In [ ]:
item_dfs = load_dfs()

In [ ]:
len(item_dfs)

In [ ]:
type(item_dfs)

In [ ]:
cleaned_item_names[0]

In [ ]:
for item_df in item_dfs:
    try:
        col = item_df.columns[0]
    except:
        invalid_dfs += 1 
        print(item_df)

In [ ]:
retrieved_names = [item_df.columns[0][:-6] for item_df in item_dfs]

In [ ]:
len(retrieved_names)

In [ ]:
len(item_dfs)

In [ ]:
df = pd.DataFrame({'name': retrieved_names, 'price_df': item_dfs})

In [ ]:
df.head()

# Feature creation

In [ ]:
#item_df_training = tl_df['price_df'][300]

In [ ]:
days_back = 90

rolling_averages = [3,7,14,21,30,60,90]

days_ahead_to_predict = 4

In [ ]:
def create_predictive_features(price_df):
    item_df_training = price_df
    item_name = item_df_training.columns[0]
    for i in range(days_back):
        item_df_training[f'{item_name} {i} days ago'] = item_df_training[item_name].shift(i)
    for i in range(days_back):
        item_df_training[f'{item_name} {i} delta days ago'] = item_df_training[item_name].shift(i) - item_df_training[item_name]
    for rolling_average in rolling_averages:
        item_df_training[f'{item_name} {rolling_average} day rolling average'] = item_df_training[item_name].rolling(rolling_average).mean()
    for rolling_average in rolling_averages:
        item_df_training[f'{item_name} delta {rolling_average} day rolling average'] = item_df_training[item_name] - item_df_training[f'{item_name} {rolling_average} day rolling average']
    item_df_training['Y'] = item_df_training[item_name].shift(-1*days_ahead_to_predict)
    return item_df_training

        

In [ ]:
#price_df = tl_df['price_df'][300]

In [ ]:
def trim_df(item_df_training):
    trimmed_df = item_df_training.dropna(how='any')
    return trimmed_df
    

# SGD LR

In [ ]:
def create_model(tl_df_row):
    trimmed_df = tl_df_row['trimmed_df']
    try:
        X = trimmed_df.drop(['Y'], axis=1)
        y = trimmed_df['Y']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=False)
        regr = linear_model.LinearRegression()
        regr.fit(X_train, y_train)
        y_pred = regr.predict(X_test)
        mean_squared_error = sklearn.metrics.mean_squared_error(y_test, y_pred)
        r2_score = sklearn.metrics.r2_score(y_test, y_pred)
        return (regr, y_pred, y_test, mean_squared_error, r2_score)
    except:
        return (0, 0, 0, 0, 0)

In [ ]:
#plt.scatter(y_test, y_pred)

# "Backtest"

In [ ]:
starting_cash = 10**7

In [ ]:
slippage = 0
thresh_buy = 0.00
thresh_sell = 0
max_volume_buy = 0.01

In [ ]:
def should_buy(row):
    if row['perc predicted increase'] > thresh_buy:
        return True

def should_sell(row):
    if row['perc predicted increase'] < thresh_sell:
        return True
    
def buy_price(row):
    return row['current_price']

def sell_price(row):
    return row['current_price']

def max_buy(row):
    d_stock = max(
        min(
        (row['starting_cash']//buy_price(row)),
        math.floor(row['volume']*max_volume_buy)
        ),
        0
    )
    d_cash = -1 * d_stock * buy_price(row)
    return (d_stock, d_cash)

def max_sell(row):
    d_stock = -1 * max(
        0,
        min(row['starting_stock'],
        math.floor(row['volume'] * max_volume_buy)
        )
    )
    d_cash = d_stock *-1 * sell_price(row)
    return (d_stock, d_cash)

In [ ]:
def get_ending_cash(record_df):
    last_row = record_df.iloc[-1,:]
    cash = last_row['ending_cash']
    cash += sell_price(last_row) * last_row['ending_stock']
    return cash

In [ ]:
def create_backtest_df(tl_df_row):
    try:
        y_pred = tl_df_row['y_pred']
        y_test = tl_df_row['y_test']
        item_df_training = tl_df_row['item_df_training']
        item_name = tl_df_row['name']

        bt_df = pd.concat([pd.Series(y_pred, index= y_test.index, name='Prediction'), y_test.rename(index='Y')], axis=1)
        bt_df['Y + 1'] = bt_df['Y'].shift(-1)
        bt_df['Y - 1'] = bt_df['Y'].shift(1)


        bt_df = bt_df.fillna(method='ffill').fillna(method='bfill')

        volume_column = f'{item_name} volume'
        volume = item_df_training[volume_column]

        bt_df = bt_df.join(item_df_training[volume_column].rename(index='volume'))
        bt_df = bt_df.join(item_df_training[f'{item_name} price'].rename(index='current_price'))
        bt_df['perc predicted increase'] = (bt_df['Prediction'] - bt_df['current_price'])/bt_df['current_price']

        bt_df['starting_stock'] = 0
        bt_df['starting_cash'] = 0
        bt_df['ending_stock'] = 0
        bt_df['ending_cash'] = 0

        bt_df.loc[bt_df.index[0], 'starting_cash'] = starting_cash
        bt_df.loc[bt_df.index[0], 'ending_cash'] = starting_cash

        bt_df['action'] = ''

        copy_df = bt_df.copy()
        record_df = bt_df.copy()

        first_row = True
        for i, row in copy_df.iterrows():
            current_row = row
            if first_row:
                prev_row = current_row
                first_row = False
                continue
            current_row['starting_cash'] = prev_row['ending_cash']
            current_row['starting_stock'] = prev_row['ending_stock']
            cash = current_row['starting_cash']
            stock = current_row['starting_stock']
            if should_buy(current_row):
                d_stock, d_cash = max_buy(current_row)
                cash += d_cash
                stock += d_stock
                current_row['action'] = f'Buying + {d_stock} stock, - {d_cash} cash'
            elif should_sell(row):
                d_stock, d_cash = max_sell(current_row)
                cash += d_cash
                stock += d_stock
                current_row['action'] = f'Selling + {d_stock} stock, + {d_cash} cash'
            else:
                current_row['action'] = 'hold'
            current_row['ending_cash'] = cash
            current_row['ending_stock'] = stock
            record_df.loc[i, :] = current_row
            prev_row = current_row
        roi = get_ending_cash(record_df)/starting_cash
        last_perc_prediction = bt_df
        return (roi, record_df)
    except:
        return (0, 0)


# Predict for right now

In [ ]:
def make_predictions(tl_df_row):
    try:
        item_name = tl_df_row['name']
        regr = tl_df_row['regr']
        item_df_training = tl_df_row['item_df_training']
        current_price = item_df_training.loc[item_df_training.index[-1], f'{item_name} price']
        pred_price = regr.predict(np.array(item_df_training.drop('Y', axis=1).loc[item_df_training.index[-1],:]).reshape(1, -1))[0]
        perc_pred_increase = (pred_price - current_price)/(current_price)
        last_day = item_df_training.index[-1]
        return (perc_pred_increase, last_day, current_price)
    except:
        return(0, 0, 0)

# Filter and find trades

In [ ]:
r2_score_thresh = .85
roi_thresh = 1.05
perc_pred_increase_thresh = 0.03

In [ ]:
batch_size = 100


In [ ]:
df_list = []

In [ ]:
bought_names = []

In [ ]:
name = df.name.loc[df.name.str.contains('Clue')]
name

In [ ]:
#bought_names.append(name[2831])

In [ ]:
bought_names

In [17]:
bought = pd.DataFrame({'name': bought_names})
bought

NameError: name 'bought_names' is not defined

In [18]:
bought = pd.read_csv('data/bought')


In [19]:
bought['name'].values

array(['Frog%20slippers', 'Granite%20hammer', 'Black%20wizard%20hat%20(g)', 'Willow%20sapling', 'Clue%20box',
       'Dareeyak%20teleport%20(tablet)', "Zamorak%20d'hide%20shield"], dtype=object)

In [20]:
for i in range(0,df.shape[0],batch_size):
    tl_df = df.iloc[i:i+100,:] 
    tl_df['item_df_training'] = tl_df['price_df'].apply(create_predictive_features)

    tl_df['trimmed_df'] = tl_df['item_df_training'].apply(trim_df)
    

    tl_df['regr'], tl_df['y_pred'], tl_df['y_test'], tl_df['mean_squared_error'], tl_df['r2_score'] = zip(*tl_df.apply(create_model, axis=1))


    tl_df['roi'], tl_df['record_df'] = zip(*tl_df.apply(create_backtest_df, axis=1))


    tl_df.sort_values(by='roi', ascending = False)

    tl_df['perc_pred_increase'], tl_df['last_day'], tl_df['current_price'] = zip(*tl_df.apply(make_predictions, axis=1))


    filter_tl_df = tl_df.copy()

    filter_tl_df = filter_tl_df.dropna(how='any')

    filter_tl_df =filter_tl_df.loc[filter_tl_df['r2_score'].apply(lambda x: type(x)) != list]

    filter_tl_df = filter_tl_df.loc[filter_tl_df['r2_score'] > r2_score_thresh]

    print(filter_tl_df.shape)

    filter_tl_df = filter_tl_df.loc[filter_tl_df['roi'] > roi_thresh]

    print(filter_tl_df.shape)

    filter_tl_df = filter_tl_df.loc[
            (filter_tl_df['perc_pred_increase'] > perc_pred_increase_thresh) \
        | (filter_tl_df['name'].isin(bought['name']))
                                   ]

    print(filter_tl_df.shape)

    print(filter_tl_df)
    df_list.append(filter_tl_df)

NameError: name 'df' is not defined

In [58]:
filter_tl_df

,name,price_df,item_df_training,trimmed_df,regr,y_pred,y_test,mean_squared_error,r2_score,roi,record_df,perc_pred_increase,last_day,current_price


In [59]:
df_list

[                name                                           price_df  \
 14  Wildblood%20seed              Wildblood%20seed price  Wildblood%...   
 
                                      item_df_training  \
 14              Wildblood%20seed price  Wildblood%...   
 
                                            trimmed_df                regr  \
 14              Wildblood%20seed price  Wildblood%...  LinearRegression()   
 
                                                y_pred  \
 14  [140.19508315409934, 146.8516792931727, 147.25...   
 
                                                y_test  mean_squared_error  \
 14  date_obj
 2020-04-17    143.0
 2020-04-18    148...           79.731955   
 
     r2_score       roi                                          record_df  \
 14   0.93529  1.082853              Prediction      Y  Y + 1  Y - 1   ...   
 
     perc_pred_increase    last_day  current_price  
 14            0.090415  2020-08-19           95.0  ,
                 name      

In [60]:
trading_df = pd.concat(df_list, axis=0).sort_values(by='perc_pred_increase', ascending=False)
trading_df.columns

Index(['name', 'price_df', 'item_df_training', 'trimmed_df', 'regr', 'y_pred',
       'y_test', 'mean_squared_error', 'r2_score', 'roi', 'record_df',
       'perc_pred_increase', 'last_day', 'current_price'],
      dtype='object')

In [61]:
trading_df['pred_price'] = (trading_df['perc_pred_increase'] + 1) * trading_df['current_price']

In [62]:
#trading_df.loc[:,['name', 'mean_squared_error', 'r2_score', 'roi', 'perc_pred_increase','last_day', 'current_price']]

In [63]:
to_buy = trading_df.loc[~trading_df['name'].isin(bought['name'])].loc[:,['name', 'mean_squared_error', 'r2_score', 'roi', 'perc_pred_increase','last_day', 'current_price', 'pred_price']]
to_buy

,name,mean_squared_error,r2_score,roi,perc_pred_increase,last_day,current_price,pred_price
2682,Willow%20seed,6.394325e+04,0.917992,1.104406,0.131213,2020-08-19,1984.0,2.244327e+03
1475,Shaman%20mask,2.181453e+06,0.916722,1.079189,0.125055,2020-08-19,9605.0,1.080616e+04
14,Wildblood%20seed,7.973196e+01,0.935290,1.082853,0.090415,2020-08-19,95.0,1.035895e+02
2790,Oak%20logs,1.018821e+02,0.931330,6.220320,0.063589,2020-08-19,66.0,7.019685e+01
1474,Musketeer%20hat,3.961510e+07,0.916067,1.173366,0.035390,2020-08-19,165651.0,1.715133e+05
1508,Saradomin%20godsword%20ornament%20kit,7.220791e+10,0.894929,1.293358,0.031916,2020-08-19,5580814.0,5.758931e+06


In [64]:
to_sell = trading_df.loc[trading_df['name'].isin(bought['name'])].loc[:,['name', 'mean_squared_error', 'r2_score', 'roi', 'perc_pred_increase','last_day', 'current_price', 'pred_price']]
to_sell

,name,mean_squared_error,r2_score,roi,perc_pred_increase,last_day,current_price,pred_price
2130,Granite%20hammer,1.185856e+09,0.947357,1.412291,0.075942,2020-08-19,329222.0,354223.865385
280,Dareeyak%20teleport%20(tablet),3.807419e+05,0.903465,1.277237,0.064452,2020-08-19,8319.0,8855.176401
2831,Clue%20box,3.435254e+07,0.854827,1.473612,0.049831,2020-08-19,101553.0,106613.482865
3027,Zamorak%20d'hide%20shield,2.097375e+08,0.880026,1.061663,0.038858,2020-08-19,280094.0,290977.949074
2986,Willow%20sapling,7.020167e+04,0.918549,1.088280,0.033328,2020-08-19,2398.0,2477.920020
1868,Black%20wizard%20hat%20(g),1.064399e+08,0.945345,1.479078,0.016905,2020-08-19,338836.0,344564.171875
180,Frog%20slippers,3.314156e+07,0.972149,1.104523,-0.088115,2020-08-19,117550.0,107192.068087



# Update with actual trades

In [1]:
bought

NameError: name 'bought' is not defined

In [85]:
bought

,name
0,Frog%20slippers
1,Granite%20hammer
2,Black%20wizard%20hat%20(g)
3,Willow%20sapling
4,Clue%20box


In [84]:
currently_buying = to_buy['name'][0:2]
currently_buying


280     Dareeyak%20teleport%20(tablet)
3027         Zamorak%20d'hide%20shield
Name: name, dtype: object

In [86]:
bought.merge(currently_buying, how='outer')

,name
0,Frog%20slippers
1,Granite%20hammer
2,Black%20wizard%20hat%20(g)
3,Willow%20sapling
4,Clue%20box
5,Dareeyak%20teleport%20(tablet)
6,Zamorak%20d'hide%20shield


In [87]:

bought = bought.merge(currently_buying, how='outer')

In [88]:
#bought = bought.drop([0,2,3])

In [89]:
#bought.loc[-1] = 'Bottomless%20compost%20bucket'

In [90]:
bought

,name
0,Frog%20slippers
1,Granite%20hammer
2,Black%20wizard%20hat%20(g)
3,Willow%20sapling
4,Clue%20box
5,Dareeyak%20teleport%20(tablet)
6,Zamorak%20d'hide%20shield


In [91]:
#bought.merge(currently_buying)
bought.to_csv('data/bought', index=False)

In [92]:
pd.read_csv('data/bought')

,name
0,Frog%20slippers
1,Granite%20hammer
2,Black%20wizard%20hat%20(g)
3,Willow%20sapling
4,Clue%20box
5,Dareeyak%20teleport%20(tablet)
6,Zamorak%20d'hide%20shield


In [93]:
trading_

NameError: name 'trading_' is not defined

In [94]:
trading_df.loc[:,['name', 'mean_squared_error', 'r2_score', 'roi', 'perc_pred_increase','last_day']].to_csv('data/trading_8.2.csv')

# fastai tabular data

In [ ]:
procs = [Normalize]

In [ ]:
valid_idx = range(int(len(df)*.8), len(df)-60)

In [ ]:
dep_var = 'Y'

In [ ]:
path = Path(f'./data/wiki_urls/{item_name}')

In [ ]:
trimmed_df['date_obj'] = trimmed_df.index

In [ ]:
trimmed_df['date_obj'] = trimmed_df['date_obj'].apply(lambda x: pd.to_datetime(x))

In [ ]:
time_trimmed_df = trimmed_df.loc[trimmed_df['date_obj'] > pd.to_datetime('2018-05-01')]

In [ ]:
time_trimmed_df[item_name].plot()

In [ ]:
time_trimmed_df = time_trimmed_df.drop(columns=['date_obj'], axis=1)

In [ ]:
trimmed_df = time_trimmed_df

In [ ]:
df = trimmed_df

In [ ]:
df

In [ ]:
data = TabularDataBunch.from_df(path, df, dep_var, valid_idx=valid_idx, procs=procs, cat_names=[])

In [ ]:
learn = tabular_learner(data, layers=[500,100], emb_szs={'native-country': 10}, metrics=mean_squared_error)

In [ ]:
data.show_batch()

In [ ]:
learn.save('stage-0')

In [ ]:
learn.lr_find(end_lr=1e3)

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, max_lr=1e-2)

In [ ]:
learn.fit_one_cycle(5, max_lr=1e-2)

In [ ]:
learn.show_results(ds_type=DatasetType.Train)

In [ ]:
learn.save('stage-1')

In [ ]:
learn = learn.load('stage-1')

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, 3e-5)

In [ ]:
learn.show_results(ds_type=DatasetType.Train)

In [ ]:
learn.save('stage-2')

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, 3e-6)

In [ ]:
learn.save('stage-3')


In [ ]:
learn.show_results(ds_type=DatasetType.Train)

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, 3e-4)

In [ ]:
learn.save('stage-4')

In [ ]:
learn.lr_find(end_lr=1e2)
learn.recorder.plot()

In [ ]:
learn.show_results(ds_type=DatasetType.Train)

In [ ]:
learn.fit_one_cycle(10, 1e-4)

In [ ]:
learn = learn.load('stage-4')

In [ ]:
learn.show_results(ds_type=DatasetType.Train)

# Plot results

In [ ]:
int(learn.predict(df.iloc[0])[1])

In [ ]:
learn.predict(df.iloc[0])

In [ ]:
plot_x = [float(learn.predict(df.iloc[i])[1]) for i in range(len(df))]

In [ ]:
plot_y = df['Y']

In [ ]:
plot_y

In [ ]:
plot_x

In [ ]:
plot_df = pd.DataFrame(plot_y)

In [ ]:
plot_df['x'] = plot_x

In [ ]:
plot_df.head()

In [ ]:
plot_df.plot()

In [ ]:
plt.plot(plot_x)

In [ ]:
plt.plot(plot_y)

In [ ]:
plt.plot(range(len(plot_x)),np.array([plot_x,plot_y]).T)

In [ ]:
len(plot_x)

In [ ]:
plt.plot(range(len(plot_x[len(plot_x)-60:])), np.array([plot_x[len(plot_x)-60:], plot_y[len(plot_y)-60:]]).T)
plt.legend()

In [ ]:
plt.plot(plot_x[len(plot_x)-60:])

In [ ]:
len(plot_x)

In [ ]:
plt.plot(plot_y[len(plot_y)-60:])

In [ ]:
x_series = pd.Series(plot_x, name='Predict')

In [ ]:
type(x_series)

In [ ]:
type(plot_y)

In [ ]:
y_series = plot_y.reindex(x_series.index)

In [ ]:
y_series

In [ ]:
x_series

In [ ]:
pd.concat([y_series, x_series],  axis=1).iloc[1705-60:].plot()

In [ ]:
learn.drop

In [ ]:
learn.save('stage-final')
learn.export()

# Load in the full data (recent rows will have been dropped because Ys were N/A)

In [ ]:
prediction_df = item_df_training

In [ ]:
prediction_df['constant'] = time_trimmed_df['Y'].mean() *scaling_factor

In [ ]:
prediction_df.columns

In [ ]:
prediction_df = prediction_df.fillna(method='ffill').fillna(method='bfill').fillna(0)

In [ ]:
predict_data = TabularDataBunch.from_df(path, prediction_df, dep_var, valid_idx=valid_idx, procs=procs, cat_names=[])

In [ ]:
predict_learn = tabular_learner(data, layers=[500,100], emb_szs={'native-country': 10}, metrics=mean_squared_error)

In [ ]:
predict_learn =predict_learn.load('stage-final')

In [ ]:
predict_learn.show_results(ds_type=DatasetType.Train)

In [ ]:
predict_learn.predict(prediction_df.iloc[0])

In [ ]:
predict_learn.predict(prediction_df.iloc[100])

In [ ]:
prediction_df

In [ ]:
predictions = [float(predict_learn.predict(prediction_df.iloc[i])[1]) for i in range(len(prediction_df))]


In [ ]:
predictions

In [ ]:
prediction_df.tail()

In [ ]:
prediction_df['predictions'] = predictions

In [ ]:
prediction_df['predictions'].tail(30)

In [ ]:
prediction_df

#### 